In [33]:
import json
import glob
from datetime import datetime
import spacy
from collections import defaultdict

In [34]:
# Get all files stating with 'reviews' and ending with '.json'
reviews_files = glob.glob('../data/Takeout/**/reviews*.json', recursive=True)

# Load and process each review file
reviews = []
for file_path in reviews_files:
    with open(file_path, 'r') as f:
        data = json.load(f)
        reviews.extend(data.get("reviews", []))

print(f"Total reviews loaded: {len(reviews)}")


Total reviews loaded: 1148


In [35]:
filtered_year = 2025
filtered_month = 8
filtered_stars = ['FOUR', 'FIVE']  # Example star ratings to filter

In [36]:
filtered_reviews = []

for review in reviews:
    update_time = review.get("updateTime")
    if update_time:
        try:
            dt = datetime.fromisoformat(update_time.replace('Z', '+00:00'))
            start_rating = review.get("starRating", "").upper()
            if start_rating not in filtered_stars:
                continue
            if dt.year == filtered_year and dt.month == filtered_month:
                filtered_reviews.append(review)
        except Exception as e:
            print(f"Skipping review with invalid date: {update_time}, error: {e}")


print(f"Reviews from {filtered_year}-{filtered_month:02d}: {len(filtered_reviews)}")

Reviews from 2025-08: 110


In [37]:
for r in filtered_reviews:
    name = r.get("reviewer", {}).get("displayName", "Anonymous")
    stars = r.get("starRating", "N/A")
    comment = r.get("comment", "")
    time = r.get("updateTime")
    print(f"{time} | {name} | {stars} stars\n{comment}\n{'-'*60}")

2025-08-21T00:16:25.554263Z | Antoinnette Kpange | FIVE stars
Jack and Hannah are awesome!! Thank you so much for the wonderful service 😊 Will be back 🥰
------------------------------------------------------------
2025-08-20T21:08:09.391505Z | amelia flores | FIVE stars
Loved Hannah and her work! Her frenchies are perfection! Also fast service and a super relaxing environment!
------------------------------------------------------------
2025-08-20T19:07:21.905839Z | Kayla Slater | FIVE stars
Hanna was great !
------------------------------------------------------------
2025-08-20T08:42:47.214853Z | Angela Roddy | FIVE stars
This place is beautiful with beautiful, and welcoming staff. The level of professionalism and gratitude exceeded expectations‼️
Ask for Demi she will not disappoint, she does nails, toes and eye lashes, deliveries with expertise.
------------------------------------------------------------
2025-08-19T20:16:50.550105Z | Anita Beasley | FIVE stars

-------------------

In [38]:
# pip install spacy
# python -m spacy download en_core_web_sm

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

In [39]:
filtered_reviews[0]

{'reviewer': {'displayName': 'Antoinnette Kpange'},
 'starRating': 'FIVE',
 'comment': 'Jack and Hannah are awesome!! Thank you so much for the wonderful service 😊 Will be back 🥰',
 'createTime': '2025-08-21T00:16:25.554263Z',
 'updateTime': '2025-08-21T00:16:25.554263Z',
 'reviewReply': {'comment': 'Thank you so much! 💖 We’re so glad you enjoyed the wonderful service and can’t wait to welcome you back! 💅🌸😊🥰',
  'updateTime': '2025-09-04T04:29:06.927711Z'},
 'name': 'accounts/108145091469061579925/locations/10316758101251449667/reviews/Ci9DQUlRQUNvZENodHljRjlvT21WUVZqTjJibE5DY0Y5WldtOUdVbVF6ZFhGdVpIYxAB'}

In [45]:
name_to_review_ids = defaultdict(set)

for r in filtered_reviews:
    doc = nlp(r.get("comment", ""))
    review_id = r.get("name")
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            name_to_review_ids[ent.text.strip()].add(review_id)

# Count and print results
name_counts = {name: len(ids) for name, ids in name_to_review_ids.items()}
sorted_names = sorted(name_counts.items(), key=lambda x: -x[1])

print(f"📌 Total unique names mentioned: {len(sorted_names)}\n")
for name, count in sorted_names:
    print(f"{name}: mentioned in {count} review(s)")

📌 Total unique names mentioned: 27

Hannah: mentioned in 19 review(s)
Jack: mentioned in 8 review(s)
Tom: mentioned in 7 review(s)
Henry: mentioned in 4 review(s)
Emma: mentioned in 3 review(s)
Myke: mentioned in 3 review(s)
Duckie: mentioned in 2 review(s)
Tammy: mentioned in 2 review(s)
Kim: mentioned in 2 review(s)
Loved Hannah: mentioned in 1 review(s)
Don: mentioned in 1 review(s)
LUCY: mentioned in 1 review(s)
Celine: mentioned in 1 review(s)
Ducki: mentioned in 1 review(s)
Mike: mentioned in 1 review(s)
Tina: mentioned in 1 review(s)
hannah: mentioned in 1 review(s)
Will: mentioned in 1 review(s)
Katie: mentioned in 1 review(s)
Tony: mentioned in 1 review(s)
Nancy: mentioned in 1 review(s)
Ryan: mentioned in 1 review(s)
Hana: mentioned in 1 review(s)
Devin: mentioned in 1 review(s)
John: mentioned in 1 review(s)
Demi: mentioned in 1 review(s)
Exelente: mentioned in 1 review(s)
